# Lifetime PD Model POC

## Introduction

## Setup

### Initialize the client library

In [115]:

import validmind as vm

vm.init(
  api_host = "http://localhost:3000/api/v1/tracking",
  api_key = "...",
  api_secret = "...",
  project = "..."
)

2023-08-15 13:30:55,200 - INFO(validmind.api_client): Connected to ValidMind. Project: [9] Credit Risk Scorecard - Initial Validation (cllaz74gb067dszy6donpqm98)
INFO: Connected to ValidMind. Project: [9] Credit Risk Scorecard - Initial Validation (cllaz74gb067dszy6donpqm98)


### Import Libraries

In [116]:
from notebooks.probability_of_default.helpers.Developer import Developer
from notebooks.probability_of_default.helpers.scorecard_tasks import *
from notebooks.probability_of_default.helpers.model_development_tasks import *

### Input Parameters

In [117]:
default_column = 'default'

### Load Credit Risk Scorecard

In [119]:
developer = Developer()
scorecard = developer.load_objects_from_pickle("datasets/scorecard_data_and_models.pkl")

df_train_feateng = scorecard["df_train_feateng"]
df_test_feateng = scorecard["df_test_feateng"]

model_fit_final = scorecard["model_fit_final"]

INFO: Loaded 7 objects from datasets/scorecard_data_and_models.pkl


### Create ValidMind Model

In [120]:
from validmind.vm_models.test_context import TestContext

vm_df_train = vm.init_dataset(
    dataset=df_train_feateng, 
    target_column=default_column)
vm_df_test = vm.init_dataset(
    dataset=df_test_feateng, 
    target_column=default_column)

vm_model_fit_final = vm.init_model(
    model = model_fit_final, 
    train_ds=vm_df_train, 
    test_ds=vm_df_test)

test_context_models_fit_final = TestContext(models = [vm_model_fit_final])

2023-08-15 13:30:55,722 - INFO(validmind.client): Pandas dataset detected. Initializing VM Dataset instance...
INFO: Pandas dataset detected. Initializing VM Dataset instance...
2023-08-15 13:30:56,336 - INFO(validmind.client): Pandas dataset detected. Initializing VM Dataset instance...
INFO: Pandas dataset detected. Initializing VM Dataset instance...


## Data Description

### Macroeconomic Data

**Target Variable**

**DRSFRMACBS (Delinquency Rate on Single-Family Residential Mortgages, Booked in Domestic Offices, All Commercial Banks)**: This stands for Delinquency Rate on Single-Family Residential Mortgages, Booked in Domestic Offices, All Commercial Banks. It reflects the percentage of loans that are past due.

Why is the delinquency rate a good target variable for building a lifetime PD and ECL Models? 
  
- Measure of credit risk: Delinquency rate directly captures the proportion of borrowers who are behind on their payments. It's a straightforward and intuitive measure of credit risk.
- Relevance to ECL: ECL requires a forward-looking assessment of credit risk. Delinquencies can provide early warning signals about loans that might eventually result in credit losses, making it directly relevant to ECL modeling.

**Features**

- **GDPC1 (Real Gross Domestic Product)**: Economic downturns, indicated by shrinking GDP, can lead to an increase in loan delinquencies as borrowers may face financial difficulties. A growing economy, on the other hand, may correlate with fewer delinquencies.

- **UNRATE (U.S. Unemployment Rate)**: A rise in unemployment rates usually correlates with an increase in delinquencies. When people lose jobs, they may have difficulty meeting financial obligations, including loan payments.

- **MORTGAGE30US (30-year fixed rate mortgage average)**: The interest rate environment can have an influence on the propensity for delinquencies, especially for adjustable-rate loans. High-interest rates can lead to larger monthly payments, increasing the chances of delinquency for some borrowers.

- **CPIAUCSL (Consumer Price Index for All Urban Consumers)**: Inflation can erode purchasing power, making it more challenging for borrowers to meet their debt obligations.

- **FEDFUNDS (Effective federal funds rate)**: The short-term interest rate can impact borrowing costs. It might indirectly influence delinquency rates, especially if borrowers are sensitive to changes in their loan rates or if they have loans with variable rates.

- **GS3, GS5, GS10 (Treasury constant maturity rates)**: These rates can serve as a proxy for the broader interest rate environment. They can influence both the borrowing cost and the appetite of financial institutions to lend. Fluctuations in these rates can potentially impact delinquency rates.

- **CSUSHPISA (S&P/Case-Shiller U.S. National Home Price Index)**: For mortgage loans, changes in home values can play a significant role. Borrowers are more likely to default on a mortgage if the value of the underlying property falls below the loan amount.

In [121]:
from validmind.tests.data_validation.TimeSeriesOutliers import TimeSeriesOutliers

vm_df = vm.init_dataset(
    dataset=df_macro_micro_raw, 
    target_column=macro_to_micro_target_column)

test_context = TestContext(dataset=vm_df)

params = {"zscore_threshold": 3}

metric = TimeSeriesOutliers(test_context, params)
metric.run()
await metric.result.log()
metric.result.show()

2023-08-15 13:30:56,569 - INFO(validmind.client): Pandas dataset detected. Initializing VM Dataset instance...
INFO: Pandas dataset detected. Initializing VM Dataset instance...
INFO: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
INFO: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
INFO: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
INFO: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
INFO: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
INFO: N

In [122]:
from validmind.tests.data_validation.TimeSeriesMissingValues import TimeSeriesMissingValues

params = {"min_threshold": 2}

metric = TimeSeriesMissingValues(test_context, params)
metric.run()
await metric.result.log()
metric.result.show()

In [123]:
from validmind.tests.data_validation.TimeSeriesFrequency import TimeSeriesFrequency

metric = TimeSeriesFrequency(test_context)
metric.run()
await metric.result.log()
metric.result.show()

### GLM Logistic Regression Model

In [124]:
print(model_fit_final.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                default   No. Observations:               109746
Model:                            GLM   Df Residuals:                   109732
Model Family:                Binomial   Df Model:                           13
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -47702.
Date:                Tue, 15 Aug 2023   Deviance:                       95403.
Time:                        13:31:10   Pearson chi2:                 1.10e+05
No. Iterations:                     5   Pseudo R-squ. (CS):            0.08209
Covariance Type:            nonrobust                                         
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                     

In [125]:
from validmind.tests.model_validation.statsmodels.RegressionModelsCoeffs import RegressionModelsCoeffs

metric = RegressionModelsCoeffs(test_context_models_fit_final)
metric.run()
await metric.result.log()
metric.result.show()

In [126]:
from validmind.tests.model_validation.statsmodels.RegressionCoeffsPlot import RegressionCoeffsPlot

metric = RegressionCoeffsPlot(test_context_models_fit_final)
metric.run()
await metric.result.log()
metric.result.show()

## Data Preparation

### Macroeconomic Data

In [127]:
# Remove COVID years to avoid outliers 
df_macro_micro_filtered = df_macro_micro_raw[df_macro_micro_raw.index <= '2019-12-31']

# Sample frequencies to Monthly
resampled_df = df_macro_micro_filtered.resample("QS-OCT").last()

# Remove all missing values
nona_df = resampled_df.dropna()

# Take the first different across all variables
preprocessed_df = nona_df.diff().dropna()

In [128]:
vm_df = vm.init_dataset(
    dataset=preprocessed_df, 
    target_column=macro_to_micro_target_column)

test_context = TestContext(dataset=vm_df)

params = {"min_threshold": 2}

metric = TimeSeriesMissingValues(test_context, params)
metric.run()
await metric.result.log()
metric.result.show()

2023-08-15 13:31:12,485 - INFO(validmind.client): Pandas dataset detected. Initializing VM Dataset instance...
INFO: Pandas dataset detected. Initializing VM Dataset instance...


In [129]:
metric = TimeSeriesFrequency(test_context)
metric.run()
await metric.result.log()
metric.result.show()

## Exploratory Data Analysis

In [130]:
from validmind.tests.data_validation.TimeSeriesLinePlot import TimeSeriesLinePlot

metric = TimeSeriesLinePlot(test_context)
metric.run()
await metric.result.log()
metric.result.show()

In [131]:
from validmind.tests.data_validation.LaggedCorrelationHeatmap import LaggedCorrelationHeatmap

metric = LaggedCorrelationHeatmap(test_context)
metric.run()
await metric.result.log()
metric.result.show()

In [132]:
from validmind.tests.data_validation.EngleGrangerCoint import EngleGrangerCoint

metric = EngleGrangerCoint(test_context)
metric.run()
await metric.result.log()
metric.result.show()

## Feature Selection

In [133]:
feature_selection_df = preprocessed_df[macro_to_micro_preliminary_features + macro_to_micro_target_column]

## Model Training 

In [134]:
import statsmodels.api as sm

# Split the data into predictors and target
X = feature_selection_df.drop(columns=macro_to_micro_target_column)
y = feature_selection_df[macro_to_micro_target_column]

# Add a constant to the predictors
X = sm.add_constant(X)

# Fit the OLS model
model = sm.OLS(y, X).fit()

# Print the summary statistics of the regression model
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:             DRSFRMACBS   R-squared:                       0.521
Model:                            OLS   Adj. R-squared:                  0.494
Method:                 Least Squares   F-statistic:                     19.58
Date:                Tue, 15 Aug 2023   Prob (F-statistic):           2.36e-15
Time:                        13:31:30   Log-Likelihood:                 4.3301
No. Observations:                 115   AIC:                             5.340
Df Residuals:                     108   BIC:                             24.55
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0780      0.044      1.760      0.0

In [137]:
final_features = ['UNRATE', 'FEDFUNDS', 'CSUSHPISA']

final_features_df = feature_selection_df[final_features + macro_to_micro_target_column]

# Split the data into predictors and target
X = final_features_df.drop(columns=macro_to_micro_target_column)
y = final_features_df[macro_to_micro_target_column]

# Add a constant to the predictors
X = sm.add_constant(X)

# Fit the OLS model
model = sm.OLS(y, X).fit()

# Print the summary statistics of the regression model
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:             DRSFRMACBS   R-squared:                       0.520
Model:                            OLS   Adj. R-squared:                  0.507
Method:                 Least Squares   F-statistic:                     40.06
Date:                Tue, 15 Aug 2023   Prob (F-statistic):           1.27e-17
Time:                        13:33:05   Log-Likelihood:                 4.1924
No. Observations:                 115   AIC:                           -0.3848
Df Residuals:                     111   BIC:                             10.59
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0864      0.026      3.313      0.0